## EE-361M Introduction to Data Mining
## Assignment #5
## Due: Thursday, Apr 14, 2016 by 2pm; Total points: 60


Your homework should be written in a **Jupyter notebook** (if this isn't possible, let me know). Please use this naming format for your notebook you submit: **Group(Group Num)_HW(HW Number).ipynb**. For example, Group1_HW1.ipynb. Homeworks should be submitted through Canvas in your **groups of 3 from the first homework**. If groups need to be adjusted please contact the TA. Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting.

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pydataset import data
from sklearn import tree, svm, grid_search
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

%matplotlib inline

# Question 1 (1+1+5+3+3+2=15pts) - Logistic Regression

In this question we will be predicting whether someone will have an affair! Yes - there is data on this. See below on how to import the data.
1. Convert naffairs to a binary variable hadAffair which is 1 if had an affair and zero otherwise
2. Split the data into training and test. Use 42 as random seed and use 1/3rd of the data for testing. Our y variable is hadAffair and X matrix includes all the other variables except naffairs.
3. Train a logistic regression with almost no regularization (pass l2 (ridge) to penalty and 1,000 to the C parameter which is the inverse of regularization strength lambda. This essentially does l2 regularization but applies very little weight to the penalty term) and report the confusion matrix on the test data. Also report the accuracy for the "no affairs" class, the affairs class, and the average per-class accuracy on the test data. Average per-class accuracy is described in this [post](http://blog.dato.com/how-to-evaluate-machine-learning-models-part-2a-classification-metrics).
4. Repeat step 3 except use l2 penalty with Cs of [.001, .01,0.1, 1]. You will want to use k-fold cross validation to select the best parameter. To evaluate which parameter is best, maximize the average per-class accuracy. To help with this task, check out [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and how to make your own [custom scorer](http://scikit-learn.org/stable/modules/model_evaluation.html).
5. Repeat question 4 except use l1 (i.e. Lasso) instead of l2 as the penalty type.
6. Which model produces the best average per-class accuracy? Why do you think this is the case? How do the models handle the different classes, and why is this so?

In [87]:
from pydataset import data
from sklearn.cross_validation import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.grid_search import GridSearchCV

df = data('affairs')

# Part 1
df.loc[:,'hadAffair'] = pd.Series(0, index=df.index)
for index, row in df.iterrows():
    if (df.ix[index,'naffairs']>0):
        df.set_value(index,'hadAffair',1)
    else:
        df.set_value(index,'hadAffair',0)
        
# Part 2
y = df.hadAffair
X = df.drop(["hadAffair", "naffairs"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33333, random_state=42)

# Part 3
def average_class_acc(actual, predict):
    confMatrix = confusion_matrix(actual,predict)
    class1Acc = float(confMatrix[0,0])/(confMatrix[0,0]+confMatrix[0,1])
    class2Acc = float(confMatrix[1,1])/(confMatrix[1,1]+confMatrix[1,0])
    return (class1Acc+class2Acc)/2

def conf_matrix_analysis(y_test, y_pred):
    confMatrix = confusion_matrix(y_test, y_pred)
    print confMatrix
    noAffAcc = float(confMatrix[0,0])/(confMatrix[0,0]+confMatrix[0,1])
    print "No Affairs class accuracy: "+str(confMatrix[0,0])+"/("+str(confMatrix[0,0])+"+"+\
        str(confMatrix[0,1])+") = " + str(100.0*noAffAcc) +"%"
    affAcc = float(confMatrix[1,1])/(confMatrix[1,1]+confMatrix[1,0])
    print "Affairs class accuracy: "+str(confMatrix[1,1])+"/("+str(confMatrix[1,1])+"+"+\
        str(confMatrix[1,0])+") = " + str(100.0*affAcc) +"%"
    print "Average per class accuracy: ("+str(100.0*noAffAcc)+"+"+str(100.0*affAcc)+")/2 = " +\
        str(average_class_acc(y_test, y_pred)) +"%\n"

def conf_matrix_analysis_logReg(penalty_value, c_value):
    logReg = LogisticRegression(penalty=penalty_value, C=c_value)
    logReg = logReg.fit(X_train, y_train)
    y_pred = logReg.predict(X_test)
    print "Confusion Matrix for C = "+ str(c_value)+":"
    conf_matrix_analysis(y_test, y_pred)

print "Part 3\n"
conf_matrix_analysis_logReg('l2',1000)

#Part 4
print "Part 4: l2 Penalty\n"

avg_acc = make_scorer(average_class_acc, greater_is_better=True)
logReg = LogisticRegression()
params = {'penalty':['l2'], 'C':[.001, .01,0.1, 1]}
gscv = GridSearchCV(logReg, params, scoring=avg_acc)
gscv = gscv.fit(X_train, y_train)
print "Confusion Matrix"
conf_matrix_analysis(y_test, gscv.predict(X_test))
print "Best C param: " + str(gscv.best_params_['C'])
print "Average per class accuracy: " + str(gscv.score(X_test, y_test)) + "%\n"

#Part 5
print "Part 5: l1 Penalty\n"

avg_acc = make_scorer(average_class_acc, greater_is_better=True)
kf = KFold(len(X))
logReg = LogisticRegression()
params = {'penalty':['l1'], 'C':[.001, .01,0.1, 1]}
gscv = GridSearchCV(logReg, params, scoring=avg_acc, cv=kf)
gscv = gscv.fit(X, y)
print "Confusion Matrix"
conf_matrix_analysis(y_test, gscv.predict(X_test))
print "Best C param: " + str(gscv.best_params_['C'])
print "Average per class accuracy: " + str(gscv.score(X_test, y_test)) + "\n"

print "Part 6"
print "The l1 (Lasso) penalty has the same accuracy as l2 (Ridge) for the affairs class, but has better accuracy for the no "\
+"affairs   class. The lower C values seem to be too low, as the bottom 3 all predict everything as no affairs. The best C "\
"value is        actually 1000, but is 1 among the values that the GridSearchCV tested in parts 4 and 5."

Part 3

Confusion Matrix for C = 1000:
[[133  14]
 [ 41  13]]
No Affairs class accuracy: 133/(133+14) = 90.4761904762%
Affairs class accuracy: 13/(13+41) = 24.0740740741%
Average per class accuracy: (90.4761904762+24.0740740741)/2 = 0.572751322751%

Part 4: l2 Penalty

Confusion Matrix
[[139   8]
 [ 44  10]]
No Affairs class accuracy: 139/(139+8) = 94.5578231293%
Affairs class accuracy: 10/(10+44) = 18.5185185185%
Average per class accuracy: (94.5578231293+18.5185185185)/2 = 0.565381708239%

Best C param: 1
Average per class accuracy: 0.565381708239%

Part 5: l1 Penalty

Confusion Matrix
[[141   6]
 [ 44  10]]
No Affairs class accuracy: 141/(141+6) = 95.9183673469%
Affairs class accuracy: 10/(10+44) = 18.5185185185%
Average per class accuracy: (95.9183673469+18.5185185185)/2 = 0.572184429327%

Best C param: 1
Average per class accuracy: 0.572184429327

Part 6
The l1 (Lasso) penalty has the same accuracy as l2 (Ridge) for the affairs class, but has better accuracy for the no affairs   c

# Question 2 (2+3+2+3=10pts) - Support Vector Classifier

This question will continue to use the data from question 1 - including the training and test split data.
1. Fit a support vector classifier using the standard options on [sklearn's SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC). Report the confusion matrix on the test data. Also report the accuracy for the no affairs class, the affairs class, and the average per-class accuracy (same as question 1).
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
3. Briefly discuss the effect of different  C,  kernel combinations.
4. Discuss your results from parts 1 and 2 and mention how they differ from Question 1's results.

In [88]:
from sklearn.svm import SVC

def conf_matrix_analysis_svc(kernel_value, c_value):
    if(c_value==None):
        svc = SVC()
        print "Confusion Matrix for C = Default:"
    else:
        svc = SVC(kernel=kernel_value, C=c_value)
        print "Confusion Matrix for C = "+ str(c_value)+":"
    svc = svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    conf_matrix_analysis(y_test, y_pred)
    

# Part 1
print "Part 1"
conf_matrix_analysis_svc(None, None)

# Part 2
print "Part 2"
avg_acc = make_scorer(average_class_acc, greater_is_better=True)
svc = SVC()
kf = KFold(len(X), n_folds=5)
params = {'kernel':['poly', 'rbf'], 'C':[0.001, 0.01, 0.1, 1,5,10,100]}
gscv = GridSearchCV(svc, params, scoring=avg_acc, cv=kf)
gscv = gscv.fit(X, y)
print "Confusion Matrix"
conf_matrix_analysis(y_test, gscv.predict(X_test))
print "Best kernel: " + str(gscv.best_params_['kernel'])
print "Best C param: " + str(gscv.best_params_['C'])
print "Best average per class accuracy: " + str(gscv.score(X_test, y_test)) + "%\n"

# Part 3
print 'Part 3:'
print 'Poly kernel:'
conf_matrix_analysis_svc('poly', 100)
conf_matrix_analysis_svc('poly', 1)
conf_matrix_analysis_svc('poly', 10)

print 'Radial kernel:'
conf_matrix_analysis_svc('rbf', 100)
conf_matrix_analysis_svc('rbf', 1)
conf_matrix_analysis_svc('rbf', 10)

print '\nHigher C values (which correspond to lower lambda values) give better average per class accuracy. Polynomial kernel'\
+' seems to   favor the affairs class accuracy, while radial favors the no affairs class, with higher overall average per class'\
+' accuracy.'

# Part 4
print '\nPart 4:'
print "The overall average per class accuracy is close, but slightly better than the LogisticRegression in question 1."\
+" The SVC has better no affairs class accuracy than the LogisticRegression, and has the same accuracy for the affairs"\
+" class."

Part 1
Confusion Matrix for C = Default:
[[147   0]
 [ 54   0]]
No Affairs class accuracy: 147/(147+0) = 100.0%
Affairs class accuracy: 0/(0+54) = 0.0%
Average per class accuracy: (100.0+0.0)/2 = 0.5%

Part 2
Confusion Matrix
[[145   2]
 [ 44  10]]
No Affairs class accuracy: 145/(145+2) = 98.6394557823%
Affairs class accuracy: 10/(10+44) = 18.5185185185%
Average per class accuracy: (98.6394557823+18.5185185185)/2 = 0.585789871504%

Best kernel: rbf
Best C param: 100
Best average per class accuracy: 0.585789871504%

Part 3:
Poly kernel:
Confusion Matrix for C = 100:
[[129  18]
 [ 42  12]]
No Affairs class accuracy: 129/(129+18) = 87.7551020408%
Affairs class accuracy: 12/(12+42) = 22.2222222222%
Average per class accuracy: (87.7551020408+22.2222222222)/2 = 0.549886621315%

Confusion Matrix for C = 1:
[[147   0]
 [ 54   0]]
No Affairs class accuracy: 147/(147+0) = 100.0%
Affairs class accuracy: 0/(0+54) = 0.0%
Average per class accuracy: (100.0+0.0)/2 = 0.5%

Confusion Matrix for C = 10:

# Question 3 (2+1+3+1+3=10pts) - Regression Trees

This question is very similar to homework 4 question 1. Except now we will be using regression trees and not classification trees, and you will be addressing a regression problem (i.e., the independent variable "price" will not be binarized).

For this question, we will be using the housing dataset (see code below). 

1. Convert driveway, recroom, fullbase, gashw, airco, and prefarea to numeric dummy variables (1 for yes, zero for no)
2. Split the data into training and testing with a random seed of 42 and keeping 1/3rd of the data for testing
2. Fit a [decision tree regressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor) to predict price using all the data (your dummy variables plus bedrooms and bathrooms).
5. Report the root MSE on the test data.
6. How does the tree decide on a splitting point?

In [81]:
# part 1
df = data('Housing')
df_dummies = pd.get_dummies(df)
#df_dummies.head()
features = ['driveway_yes','recroom_yes', 'fullbase_yes','gashw_yes', 'airco_yes','prefarea_yes','bedrooms','bathrms']

# part 2
X = df_dummies[features].values
y = df_dummies['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#part 3
clf = DecisionTreeRegressor()
result = clf.fit(X_train, y_train)
y_pred = result.predict(X_test)
#for i in range(0,10):
#    print "y test %.2f \t | diff: " % y_test[i], y_test[i]-y_pred[i]

#part 4
print "RegTree RMSE: %.2f\n" % np.sqrt(mean_squared_error(y_test, y_pred))

#part 5
print """By selecting binary splits that minimize the sum of the squared deviations from the mean in the two 
separate partitions, i.e. in each division, all datapoints in that division correspond to a certain value"""

RegTree RMSE: 21504.49

By selecting binary splits that minimize the sum of the squared deviations from the mean in the two 
separate partitions, i.e. in each division, all datapoints in that division correspond to a certain value


# Question 4 (2+5+3=10pts) - Support Vector Regression

This question will continue to use the data from question 4 - including the training and test split data.

1. Fit a support vector regression using the standard options on [sklearn's SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Report the root MSE.
2. Repeat question 1 except use grid search to select the best value of C within this set: [0.001, 0.01, 0.1, 1,5,10,100] and try both a radial and polynomial kernel (thus trying 14 combinations). Choose the combination that minimizes MSE. Use 5 folds. Report the best model and the test root MSE.
4. Discuss your results from parts 1 and 2 and how they differ from Question 4 results.

In [85]:
#part 1
svm_reg_clf = svm.SVR()
result = svm_reg_clf.fit(X_train, y_train) 
y_pred = result.predict(X_test)
#print svm_reg_clf.score(X_test,y_test)
print "SVM Reg RMSE: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred))

#part 2
parameters = {'kernel':('poly', 'rbf'), 'C': [0.001, 0.01, 0.1, 1,5,10,100],}
grid_clf = grid_search.GridSearchCV(svm_reg_clf, parameters,cv=5)
result = grid_clf.fit(X_train, y_train)
y_pred = result.predict(X_test)
print "GridReg RMSE: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred))
print "Best Model:"
print grid_clf.best_params_

#part 3
print """Both the SVMReg and GridSearch CV RMSE scores were worse than a (unpruned!!!!) regression tree. RegTree (at least 
the last iteration I ran) produced an RMSE score of 21504.49, whereas SVMReg and GridReg produced RMSE vals of 
29005.43 and 22200.72, respectively. The lesson here is that more complex models may not always do better. """

SVM Reg RMSE: 29005.43
GridReg RMSE: 22200.72
Best Model:
{'kernel': 'poly', 'C': 100}
Both the SVMReg and GridSearch CV RMSE scores were worse than a (unpruned!!!!) regression tree. RegTree (at least 
the last iteration I ran) produced an RMSE score of 21504.49, whereas SVMReg and GridReg produced RMSE vals of 
29005.43 and 22200.72, respectively. The lesson here is that more complex models may not always do better. 


# Question 5 (3+2+2+2+2+2+2=15pts) - Random Forest

This question will also continue to use the data from Question 1.
1. Fit a random forest model grid searching over the following values: {'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}. Choose the combination that maximizes the average per-class accuracy. Use 5 folds. Report the best model, the confusion matrix, the accuracy for the no affairs class, the affairs class, and the average per-class accuracy.
2. What do the n_estimators and max_features parameters do?
3. Report the features in order of importance based on the model used in part 1
4. Repeat question 1 using an AdaBoostClassifier and grid search over the following values: {'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}
5. What does the learning_rate parameter do?
6. Report the features in order of importance based on the model used in part 4
7. Compare the results in part 1 and 4 and questions 2 and 3

In [89]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
rfc= RandomForestClassifier()
kf = KFold(len(X), n_folds=5)
params = {'n_estimators':[10, 100, 1000], 'max_features': ['auto', 'sqrt', 'log2']}
gscv = GridSearchCV(rfc, params, scoring=avg_acc, cv=kf)
gscv = gscv.fit(X, y)
print "Confusion Matrix"
conf_matrix_analysis(y_test, gscv.predict(X_test))
print "Best n estimators: " + str(gscv.best_params_['n_estimators'])
print "Best max features: " + str(gscv.best_params_['max_features'])
print "Best average per class accuracy: " + str(gscv.score(X_test, y_test)) + "%\n"

print "Part 2"
print "n_estimators gives the number of trees in the forest, while max_features determines the number of features to look for"\
"when     determining the best split based on the value of n_features.\n"

print "Part 3"
print "Feature Importances:"
f_import = gscv.best_estimator_.feature_importances_
print str(f_import)+"\n"

print "Part 4"
abc = AdaBoostClassifier()
kf = KFold(len(X), n_folds=5)
params =  {'n_estimators':[50, 500, 5000], 'learning_rate': [.001, .01, .1]}
gscv = GridSearchCV(abc, params, scoring=avg_acc, cv=kf)
gscv = gscv.fit(X, y)
print "Confusion Matrix"
conf_matrix_analysis(y_test, gscv.predict(X_test))
print "Best n estimators: " + str(gscv.best_params_['n_estimators'])
print "Best learning rate: " + str(gscv.best_params_['learning_rate'])
print "Best average per class accuracy: " + str(gscv.score(X_test, y_test)) + "%\n"

print "Part 5"
print "Learning rate weights how much each classifier contributes to the model.\n"

print "Part 6"
print "Feature Importances:"
f_import = gscv.best_estimator_.feature_importances_
print str(f_import)+"\n"

print 'Part 7'
print "The RandomForestClassifier had significantly higher average per class accuracy than any of the other types of "\
+"classifiers used in questions 1,2 or 5. This is because it has a much higher accuracy on the affair class, getting "\
+"double the correct           classifications compared to the other classifiers. The cost of this is that the RandomForest "\
+"had sligtly lower accuracy on the no affair class.\nThe AdaBoostClassifier performed on par with the other classifiers"\
+", right in between the two penalty types for                  LogisticRegression."

Confusion Matrix
[[133  14]
 [ 32  22]]
No Affairs class accuracy: 133/(133+14) = 90.4761904762%
Affairs class accuracy: 22/(22+32) = 40.7407407407%
Average per class accuracy: (90.4761904762+40.7407407407)/2 = 0.656084656085%

Best n estimators: 10
Best max features: sqrt
Best average per class accuracy: 0.656084656085%

Part 2
n_estimators gives the number of trees in the forest, while max_features determines the number of features to look forwhen     determining the best split based on the value of n_features.

Part 3
Feature Importances:
[ 0.10043227  0.05175705  0.09930904  0.04989336  0.06549995  0.08322738
  0.07227058  0.04702823  0.06698689  0.05758563  0.03965557  0.04260596
  0.03780598  0.04348614  0.04299255  0.05062891  0.04883451]

Part 4
Confusion Matrix
[[140   7]
 [ 44  10]]
No Affairs class accuracy: 140/(140+7) = 95.2380952381%
Affairs class accuracy: 10/(10+44) = 18.5185185185%
Average per class accuracy: (95.2380952381+18.5185185185)/2 = 0.568783068783%

Best n es